In [ ]:
from stock import Stock
from stock import Price
from stock import get_current_datetime
import sqlite3
from sqlite3 import Error
import yfinance as yf
import datetime
import time
import pandas as pd
import pickle
import numpy as np
from sklearn.impute import KNNImputer

In [ ]:
# connect to the DB, gather stock symbols, add to a list
# create an SQLite db connection
conn = sqlite3.connect('stockPrediction.db')

# collect stock symbols from DB, add to a list
dbq = conn.execute('SELECT stock_symbol FROM stock')
conn.commit()
dbq = dbq.fetchall()
# the result from dbq.fetchall() is a list of single-element tuples
# the list comprehension below turns it into a list of strings instead
db_symbols = [x[0] for x in dbq]

# create a dict of stock symbols and their stock_id
dbq = conn.execute('SELECT stock_id, stock_symbol FROM stock')
conn.commit()
dbq = dbq.fetchall()
stock_id_dict = dict()  # keys are ticker symbols, vals are the ID #
    
for x in dbq:
    stock_id_dict[x[1]] = x[0]

try:
    infile = open('gphl_symbol', 'rb')
    pickle_symbol = pickle.load(infile)
    pickle_symbol_read = False
except:
    pickle_symbol = None
    pickle_symbol_read = True

In [ ]:
# this bit adds the EOD price history table to the db if it does not already exist.
conn.execute('''CREATE TABLE IF NOT EXISTS eod_price_history (
    price_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    stock_id INTEGER NOT NULL,
    price_datetime TEXT NOT NULL,
    open_price REAL NOT NULL,
    high_price REAL NOT NULL,
    low_price REAL NOT NULL,
    close_price REAL NOT NULL,
    volume INTEGER NOT NULL,
    dividends REAL,
    stock_splits TEXT,
    datetime_added TEXT NOT NULL,
    FOREIGN KEY (stock_id)
        REFERENCES stock (stock_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE''')